In [ ]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 4.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import r2_score
import category_encoders as ce

## EDA

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/IEOR 142A Final Project/Data/data.xlsx')

In [ ]:
df.head()

,GpsProvider,BookingID,Market/Regular,BookingID_Date,vehicle_no,Origin_Location,Destination_Location,Org_lat_lon,Des_lat_lon,Data_Ping_time,...,TRANSPORTATION_DISTANCE_IN_KM,vehicleType,Minimum_kms_to_be_covered_in_a_day,Driver_Name,Driver_MobileNo,customerID,customerNameCode,supplierID,supplierNameCode,Material Shipped
0,CONSENT TRACK,MVCV0000927/082021,Market,2020-08-17 14:59:01.000,KA590408,"TVSLSL-PUZHAL-HUB,CHENNAI,TAMIL NADU","ASHOK LEYLAND PLANT 1- HOSUR,HOSUR,KARNATAKA","13.1550,80.1960","12.7400,77.8200",2020-08-24 00:05:09,...,320.0,NaN,NaN,NaN,NaN,ALLEXCHE45,Ashok leyland limited,VIJEXHOSR7,VIJAY TRANSPORT,BRACKET / GRAB HANDLE
1,VAMOSYS,VCV00014271/082021,Regular,2020-08-27 16:22:22.827,TN30BC5917,"DAIMLER INDIA COMMERCIAL VEHICLES,KANCHIPURAM,...","DAIMLER INDIA COMMERCIAL VEHICLES,KANCHIPURAM,...","12.8390,79.9540","12.8390,79.9540",2020-08-28 12:40:28,...,103.0,NaN,NaN,RAMESH,NaN,DMREXCHEUX,Daimler india commercial vehicles pvt lt,VJLEXSHE09,VJ LOGISTICS,ZB MODEL PLATE / 3143
2,CONSENT TRACK,VCV00014382/082021,Regular,2020-08-27 17:59:24.987,TN22AR2748,"LUCAS TVS LTD-PONDY,PONDY,PONDICHERRY","LUCAS TVS LTD-PONDY,PONDY,PONDICHERRY","11.8710,79.7390","11.8710,79.7390",2020-08-28 09:05:09,...,300.0,NaN,NaN,GIRI,NaN,LUTGCCHE06,Lucas tvs ltd,GSTEXLAK1Q,G.S. TRANSPORT,LETTERING / FUSO
3,VAMOSYS,VCV00014743/082021,Regular,2020-08-28 00:48:24.503,TN28AQ0781,"DAIMLER INDIA COMMERCIAL VEHICLES,KANCHIPURAM,...","DAIMLER INDIA COMMERCIAL VEHICLES,KANCHIPURAM,...","12.8390,79.9540","12.8390,79.9540",2020-08-28 12:40:31,...,61.0,NaN,NaN,RAVI,NaN,DMREXCHEUX,Daimler india commercial vehicles pvt lt,ARVEXNAM09,ARVINTH TRANSPORT,LU STRUT RA / RADIUS ROD
4,VAMOSYS,VCV00014744/082021,Regular,2020-08-28 01:23:19.243,TN68F1722,"LUCAS TVS LTD-PONDY,PONDY,PONDICHERRY","LUCAS TVS LTD-PONDY,PONDY,PONDICHERRY","11.8720,79.6320","11.8720,79.6320",2020-08-28 12:40:29,...,240.0,NaN,NaN,TAMIL,NaN,LUTGCCHE06,Lucas tvs ltd,SRTEXKOR96,SR TRANSPORTS,WISHBONE / V ROD/HDT


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6880 entries, 0 to 6879
Data columns (total 32 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   GpsProvider                         5927 non-null   object        
 1   BookingID                           6880 non-null   object        
 2   Market/Regular                      6880 non-null   object        
 3   BookingID_Date                      6880 non-null   datetime64[ns]
 4   vehicle_no                          6880 non-null   object        
 5   Origin_Location                     6880 non-null   object        
 6   Destination_Location                6880 non-null   object        
 7   Org_lat_lon                         6880 non-null   object        
 8   Des_lat_lon                         6880 non-null   object        
 9   Data_Ping_time                      5927 non-null   datetime64[ns]
 10  Planned_ETA             

In [ ]:
df.describe()

,BookingID_Date,Data_Ping_time,Curr_lat,Curr_lon,TRANSPORTATION_DISTANCE_IN_KM,Minimum_kms_to_be_covered_in_a_day,Driver_MobileNo
count,6880,5927,5927.000000,5927.000000,6168.000000,2820.000000,2.691000e+03
mean,2020-02-08 05:18:40.453054208,2020-04-06 04:54:24.370193920,18.679995,78.759745,553.856282,250.239362,8.598981e+09
min,2019-03-18 12:19:22,2019-06-07 18:25:10,8.166790,69.657698,0.000000,0.000000,6.000546e+09
25%,2019-10-11 18:47:32.500000,2019-12-01 07:35:10,12.871868,76.891857,40.000000,250.000000,7.651505e+09
50%,2020-01-20 13:49:40.500000,2020-06-19 16:10:08,17.470922,77.843340,160.000000,250.000000,8.930762e+09
75%,2020-07-30 13:46:34.750000128,2020-08-10 15:52:45,23.202009,80.018999,900.000000,250.000000,9.634981e+09
max,2020-12-03 13:10:21,2020-08-28 12:40:31,32.367928,95.529550,2954.700000,275.000000,1.000000e+10
std,NaN,NaN,6.075561,4.216357,758.981836,24.315485,1.131669e+09


In [ ]:
nan_percentage = (df.isna().sum() / len(df)) * 100
nan_percentage_sorted = nan_percentage.sort_values(ascending=False)
print(nan_percentage_sorted)

ontime                                62.965116
Driver_MobileNo                       60.886628
Minimum_kms_to_be_covered_in_a_day    59.011628
Driver_Name                           49.840116
delay                                 36.889535
Current_Location                      14.011628
Data_Ping_time                        13.851744
Curr_lon                              13.851744
Curr_lat                              13.851744
GpsProvider                           13.851744
vehicleType                           12.034884
TRANSPORTATION_DISTANCE_IN_KM         10.348837
trip_end_date                          2.819767
actual_eta                             0.537791
DestinationLocation_Code               0.392442
OriginLocation_Code                    0.043605
supplierNameCode                       0.000000
supplierID                             0.000000
customerNameCode                       0.000000
customerID                             0.000000
Market/Regular                         0

## Preprocessing

### Drop Unecessary Columns & Fill/Impute Missing Values

In [ ]:
df.dropna(subset = ["TRANSPORTATION_DISTANCE_IN_KM"], inplace=True)

In [ ]:
df.fillna({'GpsProvider': 'Unknown', 'Current_Location': 'Unknown', 'Driver_Name': 'Unknown', 'Driver_MobileNo': 0}, inplace=True)

In [ ]:
df.drop(['Driver_Name', 'Driver_MobileNo', 'customerID', 'customerNameCode',
         'supplierNameCode', 'DestinationLocation'], axis=1, inplace=True)

In [ ]:
df.shape

(6168, 26)

In [ ]:
df = df[~((df["ontime"] == "G") & (df["delay"] == "R"))]
df = df[~(df["ontime"].isna() & df["delay"].isna())]

### Datetime & Date Extraction

In [ ]:
df['trip_start_date'] = pd.to_datetime(df['trip_start_date'], errors='coerce')
df['trip_end_date'] = pd.to_datetime(df['trip_end_date'], errors='coerce')
df['Planned_ETA'] = pd.to_datetime(df['Planned_ETA'], errors='coerce')
df['actual_eta'] = pd.to_datetime(df['actual_eta'], errors='coerce')
df["Data_Ping_time"] = pd.to_datetime(df["Data_Ping_time"] )
df["BookingID_Date"] = pd.to_datetime(df["BookingID_Date"] )

df['Booking_Year'] = df['BookingID_Date'].dt.year
df['Booking_Month'] = df['BookingID_Date'].dt.month
df['Booking_Day'] = df['BookingID_Date'].dt.day

### Combine Delayed/Ontime -> `is_delayed`

In [ ]:
df['is_delayed'] = df['delay'].notna().astype(int)

### OHE `vehicleType` and `Market\Regular`

In [ ]:
df['vehicleType'] = df['vehicleType'].astype('category').cat.codes
df.rename(columns=lambda x: x.strip(), inplace=True)
df['Market/Regular'] = df['Market/Regular'].astype('category').cat.codes
df['GpsProvider'] = df['GpsProvider'].astype('category').cat.codes

In [ ]:
df_encoded = pd.get_dummies(df, columns = ['vehicleType', 'Market/Regular', 'GpsProvider'])

In [ ]:
date_cols = ['BookingID_Date', 'Data_Ping_time', 'Planned_ETA',
             'actual_eta', 'trip_start_date', 'trip_end_date']
for col in date_cols:
    if col in df_encoded.columns:
        df_encoded[col] = pd.to_datetime(df_encoded[col], errors='coerce').apply(lambda x: x.toordinal() if not pd.isnull(x) else None)

In [ ]:
non_numeric_cols = ['BookingID', 'Origin_Location', 'Destination_Location',
                    'Current_Location', 'Material Shipped', 'vehicle_no']
df_encoded = df_encoded.drop(columns=non_numeric_cols, errors='ignore')


In [ ]:
df_numeric = df_encoded.dropna()

In [ ]:
# Function to split coordinate pairs into two columns
def split_coordinates(df, col_name):
    lat_col = f"{col_name}_lat"
    lon_col = f"{col_name}_lon"
    df[[lat_col, lon_col]] = df[col_name].str.split(',', expand=True).astype(float)
    df.drop(columns=[col_name], inplace=True)
    return df

# Split latitude and longitude for relevant columns
coord_columns = ['Org_lat_lon', 'Des_lat_lon']
for col in coord_columns:
    if col in df.columns:
        df = split_coordinates(df, col)

# Revalidate non-numeric columns
non_numeric_cols = df.select_dtypes(include=['object']).columns.tolist()
print("Remaining non-numeric columns:", non_numeric_cols)

# Drop remaining non-numeric columns
df_numeric = df.drop(columns=non_numeric_cols, errors='ignore')

# Convert date columns to numeric (ordinal) values
date_cols = ['BookingID_Date', 'Data_Ping_time', 'Planned_ETA',
             'actual_eta', 'trip_start_date', 'trip_end_date']
for col in date_cols:
    if col in df_numeric.columns:
        df_numeric[col] = pd.to_datetime(df_numeric[col], errors='coerce').apply(lambda x: x.toordinal() if not pd.isnull(x) else None)

# Drop rows with missing values (or you can impute)
df_numeric = df_numeric.dropna()

# Ensure target column is in numeric dataset
if 'is_delayed' in df.columns and 'is_delayed' not in df_numeric.columns:
    df_numeric['is_delayed'] = df['is_delayed']

Remaining non-numeric columns: ['BookingID', 'vehicle_no', 'Origin_Location', 'Destination_Location', 'Current_Location', 'ontime', 'delay', 'OriginLocation_Code', 'DestinationLocation_Code', 'supplierID', 'Material Shipped']


In [ ]:
df_encoded.columns

Index(['BookingID_Date', 'Org_lat_lon', 'Des_lat_lon', 'Data_Ping_time',
       'Planned_ETA', 'actual_eta', 'Curr_lat', 'Curr_lon', 'ontime', 'delay',
       'OriginLocation_Code', 'DestinationLocation_Code', 'trip_start_date',
       'trip_end_date', 'TRANSPORTATION_DISTANCE_IN_KM',
       'Minimum_kms_to_be_covered_in_a_day', 'supplierID', 'Booking_Year',
       'Booking_Month', 'Booking_Day', 'is_delayed', 'vehicleType_-1',
       'vehicleType_0', 'vehicleType_1', 'vehicleType_2', 'vehicleType_3',
       'vehicleType_4', 'vehicleType_5', 'vehicleType_6', 'vehicleType_7',
       'vehicleType_8', 'vehicleType_9', 'vehicleType_10', 'vehicleType_11',
       'vehicleType_12', 'vehicleType_13', 'vehicleType_14', 'vehicleType_15',
       'vehicleType_16', 'vehicleType_17', 'vehicleType_18', 'vehicleType_19',
       'vehicleType_20', 'vehicleType_21', 'vehicleType_22', 'vehicleType_23',
       'vehicleType_24', 'vehicleType_25', 'vehicleType_26', 'vehicleType_27',
       'vehicleType_28', 

In [ ]:
df_encoded.shape

(6141, 97)

In [ ]:
df_encoded.describe()

,BookingID_Date,Data_Ping_time,Planned_ETA,actual_eta,Curr_lat,Curr_lon,trip_start_date,trip_end_date,TRANSPORTATION_DISTANCE_IN_KM,Minimum_kms_to_be_covered_in_a_day,Booking_Year,Booking_Month,Booking_Day,is_delayed
count,6141.000000,5731.000000,6141.000000,6118.000000,5731.000000,5731.000000,6141.000000,5947.000000,6141.000000,2664.000000,6141.000000,6141.000000,6141.000000,6141.000000
mean,737487.015958,737520.444251,737474.913369,737484.718535,18.666076,78.800769,737472.985344,737471.991256,555.564110,248.920796,2019.560984,7.770396,17.694838,0.679368
std,157.020697,134.526700,807.742781,808.610146,6.067758,4.244770,807.449817,819.771863,760.058385,23.782974,0.496307,2.744858,8.271848,0.466757
min,737136.000000,737217.000000,693594.000000,693594.000000,8.166790,69.657698,693594.000000,693594.000000,0.000000,0.000000,2019.000000,1.000000,1.000000,0.000000
25%,737364.000000,737394.000000,737364.000000,737383.000000,12.857474,76.951815,737364.000000,737367.000000,40.000000,250.000000,2019.000000,7.000000,11.000000,0.000000
50%,737456.000000,737594.000000,737457.000000,737501.000000,17.470922,77.875293,737456.000000,737458.000000,160.000000,250.000000,2020.000000,8.000000,19.000000,1.000000
75%,737641.000000,737648.000000,737643.000000,737648.000000,23.198477,80.019467,737641.000000,737643.000000,900.000000,250.000000,2020.000000,10.000000,25.000000,1.000000
max,737762.000000,737665.000000,737764.000000,737667.000000,32.367928,95.529550,737762.000000,737665.000000,2954.700000,275.000000,2020.000000,12.000000,31.000000,1.000000
